# SENTINEL HUD - ENTRENAMIENTO YOLOv5

Este notebook está diseñado para entrenar el modelo de detección de EPIs.
**Asegúrate de subir el archivo zip del dataset generado localmente a la sesión de Colab.**

In [ ]:
# 1. Instalar Dependencias y Clonar Repo
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
!pip install -r requirements.txt  # install

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

## 2. Configuración del Dataset (custom.yaml)
Definimos las clases y rutas.

In [ ]:
import yaml

# Definir la configuración del dataset
custom_yaml = {
    'path': '../dataset',  # dataset root dir
    'train': 'images/train',
    'val': 'images/val',
    'nc': 4,
    'names': ['Casco', 'Chaleco', 'Persona_Sin_Equipo', 'Peligro']
}

with open('custom.yaml', 'w') as f:
    yaml.dump(custom_yaml, f)

print("Archivo custom.yaml creado exitosamente.")
!cat custom.yaml

## 3. Entrenamiento
Ejecutamos el entrenamiento por 300 epochs. 
- `img 640`: Tamaño de imagen.
- `batch 16`: Batch size estándar (ajustar si falta RAM).
- `weights yolov5s.pt`: Transfer Learning desde modelo pequeño (rápido y eficiente).

In [ ]:
# Entrenar YOLOv5s en custom.yaml
!python train.py --img 640 --batch 16 --epochs 300 --data custom.yaml --weights yolov5s.pt --cache

## 4. Exportar Modelo
Descarga el archivo `best.pt` para usarlo en la aplicación local.

In [ ]:
from google.colab import files
files.download('runs/train/exp/weights/best.pt')